## Weather 
-----

Weather features for time-series forecasting

In [1]:
import pandas as pd
import requests

In [2]:
import datetime as dt
import numpy as np 
import os
import requests

from math import log10, cos, sin, asin, sqrt, exp, pi, radians

from pcse.util import ea_from_tdew, reference_ET, check_angstromAB
class NASA:
    ranges = {"LAT": (-90., 90.),
          "LON": (-180., 180.),
          "ELEV": (-300, 6000),
          "IRRAD": (0., 40e6),
          "TMIN": (-50., 60.),
          "TMAX": (-50., 60.),
          "VAP": (0.06, 199.3),  # hPa, computed as sat. vapour pressure at -50, 60 Celsius
          "RAIN": (0, 25),
          "E0": (0., 2.5),
          "ES0": (0., 2.5),
          "ET0": (0., 2.5),
          "WIND": (0., 100.),
          "SNOWDEPTH": (0., 250.),
          "TEMP": (-50., 60.),
          "TMINRA": (-50., 60.)}

    def __init__(self, latitude, longitude):
        if latitude < -90 or latitude > 90:
            msg = "Latitude should be between -90 and 90 degrees."
            raise ValueError(msg)
        if longitude < -180 or longitude > 180:
            msg = "Longitude should be between -180 and 180 degrees."
            raise ValueError(msg)

        self.MJ_to_J = lambda x: x * 1e6
        self.mm_to_cm = lambda x: x / 10.
        self.tdew_to_hpa = lambda x: ea_from_tdew(x) * 10.
        self.to_date = lambda d: d.date()
        self.HTTP_OK = 200
        self.angstA = 0.29
        self.angstB = 0.49

        self.latitude = latitude
        self.longitude = longitude

        self.power_variables = ["TOA_SW_DWN", "ALLSKY_SFC_SW_DWN", "T2M", "T2M_MIN",
                            "T2M_MAX", "T2MDEW", "WS2M", "PRECTOTCORR"]
        self._get_and_process_NASAPower(latitude, longitude)

    def _get_and_process_NASAPower(self, latitude, longitude):
            """Handles the retrieval and processing of the NASA Power data
            """
            powerdata = self._query_NASAPower_server(latitude, longitude)
            if not powerdata:
                msg = "Failure retrieving POWER data from server. This can be a connection problem with " \
                    "the NASA POWER server, retry again later."
                raise RuntimeError(msg)

            # Store the informational header then parse variables
            self.description = [powerdata["header"]["title"]]
            self.elevation = float(powerdata["geometry"]["coordinates"][2])
            
            
            df_power = self._process_POWER_records(powerdata)

            self.angstA, self.angstB = self._estimate_AngstAB(df_power)
            df_pcse = self._POWER_to_PCSE(df_power)
            
            return df_pcse
    def _query_NASAPower_server(self, latitude, longitude):
        start_date = dt.date(1990,1,1)
        end_date = dt.date.today()
        # build URL for retrieving data, using new NASA POWER api
        server = "https://power.larc.nasa.gov/api/temporal/daily/point"
        payload = {"request": "execute",
                    "parameters": ",".join(self.power_variables),
                    "latitude": latitude,
                    "longitude": longitude,
                    "start": start_date.strftime("%Y%m%d"),
                    "end": end_date.strftime("%Y%m%d"),
                    "community": "AG",
                    "format": "JSON",
                    "user": "anonymous"
                    }
        req = requests.get(server, params=payload)

        if req.status_code != self.HTTP_OK:
            msg = ("Failed retrieving POWER data, server returned HTTP " +
                    "code: %i on following URL %s") % (req.status_code, req.url)
            raise ValueError(msg)

        return req.json()

    def _process_POWER_records(self, powerdata):
        """Process the meteorological records returned by NASA POWER
        """


        fill_value = float(powerdata["header"]["fill_value"])

        df_power = {}
        for varname in self.power_variables:
            s = pd.Series(powerdata["properties"]["parameter"][varname])
            s[s == fill_value] = np.NaN
            df_power[varname] = s
        df_power = pd.DataFrame(df_power)
        df_power["DAY"] = pd.to_datetime(df_power.index, format="%Y%m%d")

        # find all rows with one or more missing values (NaN)
        ix = df_power.isnull().any(axis=1)
        # Get all rows without missing values
        df_power = df_power[~ix]

        return df_power
    
    
    def _estimate_AngstAB(self, df_power):
        """Determine Angstrom A/B parameters from Top-of-Atmosphere (ALLSKY_TOA_SW_DWN) and
        top-of-Canopy (ALLSKY_SFC_SW_DWN) radiation values.

        :param df_power: dataframe with POWER data
        :return: tuple of Angstrom A/B values

        The Angstrom A/B parameters are determined by dividing swv_dwn by toa_dwn
        and taking the 0.05 percentile for Angstrom A and the 0.98 percentile for
        Angstrom A+B: toa_dwn*(A+B) approaches the upper envelope while
        toa_dwn*A approaches the lower envelope of the records of swv_dwn
        values.
        """


        # check if sufficient data is available to make a reasonable estimate:
        # As a rule of thumb we want to have at least 200 days available
        if len(df_power) < 200:
            msg = ("Less then 200 days of data available. Reverting to " +
                   "default Angstrom A/B coefficients (%f, %f)")
            return self.angstA, self.angstB

        # calculate relative radiation (swv_dwn/toa_dwn) and percentiles
        relative_radiation = df_power.ALLSKY_SFC_SW_DWN/df_power.TOA_SW_DWN
        ix = relative_radiation.notnull()
        angstrom_a = float(np.percentile(relative_radiation[ix].values, 5))
        angstrom_ab = float(np.percentile(relative_radiation[ix].values, 98))
        angstrom_b = angstrom_ab - angstrom_a

        try:
            check_angstromAB(angstrom_a, angstrom_b)
        except Exception as e:
            msg = ("Angstrom A/B values (%f, %f) outside valid range: %s. " +
                   "Reverting to default values.")
            return self.angstA, self.angstB


        return angstrom_a, angstrom_b



    def _POWER_to_PCSE(self, df_power):

            # Convert POWER data to a dataframe with PCSE compatible inputs
            df_pcse = pd.DataFrame({"TMAX": df_power.T2M_MAX,
                                    "TMIN": df_power.T2M_MIN,
                                    "TEMP": df_power.T2M,
                                    "IRRAD": df_power.ALLSKY_SFC_SW_DWN.apply(self.MJ_to_J),
                                    "RAIN": df_power.PRECTOTCORR.apply(self.mm_to_cm), # ATTENTION
                                    "WIND": df_power.WS2M,
                                    "VAP": df_power.T2MDEW.apply(self.tdew_to_hpa),
                                    "DAY": df_power.DAY.apply(self.to_date),
                                    "LAT": self.latitude,
                                    "LON": self.longitude,
                                    "ELEV": self.elevation})
            self.df_pcse = df_pcse.reset_index(drop=True)
            return df_pcse

def ea_from_tdew(tdew):
    """
    Calculates actual vapour pressure, ea [kPa] from the dewpoint temperature
    using equation (14) in the FAO paper. As the dewpoint temperature is the
    temperature to which air needs to be cooled to make it saturated, the
    actual vapour pressure is the saturation vapour pressure at the dewpoint
    temperature. This method is preferable to calculating vapour pressure from
    minimum temperature.
    Taken from fao_et0.py written by Mark Richards
    Reference:
    Allen, R.G., Pereira, L.S., Raes, D. and Smith, M. (1998) Crop
        evapotranspiration. Guidelines for computing crop water requirements,
        FAO irrigation and drainage paper 56)
    Arguments:
    tdew - dewpoint temperature [deg C]
    """
    # Raise exception:
    if (tdew < -95.0 or tdew > 65.0):
        # Are these reasonable bounds?
        msg = 'tdew=%g is not in range -95 to +60 deg C' % tdew
        raise ValueError(msg)

    tmp = (17.27 * tdew) / (tdew + 237.3)
    ea = 0.6108 * exp(tmp)
    return ea


def check_angstromAB(xA, xB):
    """Routine checks validity of Angstrom coefficients.
    
    This is the  python version of the FORTRAN routine 'WSCAB' in 'weather.for'.
    """
    MIN_A = 0.1
    MAX_A = 0.4
    MIN_B = 0.3
    MAX_B = 0.7
    MIN_SUM_AB = 0.6
    MAX_SUM_AB = 0.9
    A = abs(xA)
    B = abs(xB)
    SUM_AB = A + B
    if A < MIN_A or A > MAX_A:
        msg = "invalid Angstrom A value!"
        raise ValueError(msg)
    if B < MIN_B or B > MAX_B:
        msg = "invalid Angstrom B value!"
        raise ValueError(msg)
    if SUM_AB < MIN_SUM_AB or SUM_AB > MAX_SUM_AB:
        msg = "invalid sum of Angstrom A & B values!"
        raise ValueError(msg)

    return [A,B]

def round_geoposition(x, prec=1, base=.5):
        return round(base * round(float(x)/base),prec)
    
    
    
def remove_outliers(df: pd.DataFrame, ranges: dict) -> pd.DataFrame: 
    """
    Replace values outside of suitable range with edge values
    
    Input: pd.DataFrame 
        weather dataframe from NASA POWER
    Output: pd.DataFrame
        weather dataframe
    
    """

    colsToCheck = ['TMAX', 'TMIN', 'IRRAD', 'RAIN', 'WIND', 'VAP']
    for col in colsToCheck:
        min_val, max_val = ranges[col]
        mask_min = df[col] < min_val
        mask_max = df[col] > max_val
        if min_val < 0: 
            df.loc[mask_min, col] = min_val * 0.8
        else:
            df.loc[mask_min, col] = min_val * 1.2
        df.loc[mask_max, col] = max_val * 0.8

    return df

def weather_loader(path_to_CSV_dir:str,
                   latitude:float,
                   longitude:float, 
                   path_to_pattern:str):
    
    path_to_save_csv_file = os.path.join(path_to_CSV_dir,
                                         f'NASA_weather_latitude_{latitude}_longitude_{longitude}.csv')
    
    if os.path.isfile(path_to_save_csv_file)==True:
        return 'Yes'
    else:

        weather = NASA(latitude, longitude)

        df_weather = weather.df_pcse
        
        # Remove outliers 
        
        df_weather = remove_outliers(df = df_weather,
                                     ranges = NASA.ranges)
        
        #create full range of dates
        r = pd.date_range(start=df_weather.DAY.min(), end=df_weather.DAY.max())
        #extend range of dates
        full_range_weather = df_weather.set_index('DAY').reindex(r).rename_axis('DAY').reset_index()
        missing_days = (full_range_weather.isna()).sum().sum()
        percent_of_missing_weather = missing_days/len(full_range_weather)*100
        filled_weather = full_range_weather.fillna(method='ffill', axis=0)

        #save as csv file
        filled_weather=filled_weather[['DAY', 'IRRAD', 'TMIN', 'TMAX', 'VAP', 'WIND', 'RAIN']]
        filled_weather['SNOWDEPTH'] = 'NaN'
        filled_weather[['IRRAD']] = filled_weather[['IRRAD']]/1000.
        filled_weather[['VAP']] = filled_weather[['VAP']]/10.
        filled_weather[['RAIN']] = filled_weather[['RAIN']]*10. # cm to mm
        filled_weather.DAY=filled_weather.DAY.dt.strftime('%Y%m%d')

        text = open(path_to_pattern, "r")
        dictReplace = {"1111":weather.longitude, 
                      "2222": weather.latitude, 
                      "3333": weather.elevation, 
                      "4444": weather.angstA, 
                      "5555": weather.angstB}
        for key, value in dictReplace.items():
                text = ''.join([i for i in text]).replace(key, str(value))
        x = open(path_to_save_csv_file,"w")
        x.writelines(text)
        x.close()
        filled_weather.to_csv(path_to_save_csv_file, mode='a', header=False, index=False)
#         weather = pcse.fileinput.CSVWeatherDataProvider(path_to_save_csv_file)
        return 'Downloaded weather from NASA system'


Platform not recognized, using system temp directory for PCSE settings.
Platform not recognized, using system temp directory for PCSE settings.


In [4]:
lat = 37.441599 
lon = -122.150545
nasa = NASA(latitude=lat, longitude=lon)

In [7]:
nasa.df_pcse.to_csv('../Datasets/weather_palalto.csv')

In [8]:
nasa.df_pcse

,TMAX,TMIN,TEMP,IRRAD,RAIN,WIND,VAP,DAY,LAT,LON,ELEV
0,11.81,5.44,7.82,3160000.0,0.734,1.81,10.053033,1990-01-01,37.441599,-122.150545,233.2
1,12.17,1.32,6.06,10900000.0,0.003,3.80,6.175001,1990-01-02,37.441599,-122.150545,233.2
2,12.66,0.88,5.53,10380000.0,0.000,1.61,6.657686,1990-01-03,37.441599,-122.150545,233.2
3,13.05,-0.27,5.44,8460000.0,0.000,1.38,6.496341,1990-01-04,37.441599,-122.150545,233.2
4,16.25,3.04,7.57,9380000.0,0.000,1.34,8.988433,1990-01-05,37.441599,-122.150545,233.2
...,...,...,...,...,...,...,...,...,...,...,...
11914,36.76,12.13,22.99,23620000.0,0.012,1.84,12.296093,2022-08-15,37.441599,-122.150545,233.2
11915,40.38,13.54,25.94,23340000.0,0.000,1.80,11.242379,2022-08-16,37.441599,-122.150545,233.2
11916,38.72,16.95,26.23,16770000.0,0.000,2.12,11.434153,2022-08-17,37.441599,-122.150545,233.2
11917,33.52,14.41,22.12,21960000.0,0.001,2.84,11.660212,2022-08-18,37.441599,-122.150545,233.2
